# import des librairies

In [262]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error

import plotly.express as px

# chargement des données

In [263]:
df = pd.read_csv("..\\data\\Walmart_Store_sales_cleaned.csv")
df.head()

,Unnamed: 0,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
0,0,6,1572117.54,0,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0
1,1,13,1807545.43,0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0
2,4,6,1644470.66,0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0
3,5,4,1857533.70,0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0
4,6,15,695396.19,0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0


# Baseline model

Pour la baseline, on va travailler avec toutes les colonnes et n'effectuer aucun stratify

In [264]:
target = "Weekly_Sales"

X = df.drop(target, axis=1) 
Y = df[target]
display(X)
display(Y)

,Unnamed: 0,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
0,0,6,0,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0
1,1,13,0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0
2,4,6,0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0
3,5,4,0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0
4,6,15,0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0
...,...,...,...,...,...,...,...,...,...,...
118,144,3,0,73.44,3.594,226.968844,6.034,2012.0,10.0,19.0
119,145,14,0,72.62,2.780,182.442420,8.899,2010.0,6.0,18.0
120,147,17,0,57.14,2.841,126.111903,NaN,2010.0,6.0,11.0
121,148,8,0,86.05,3.638,219.007525,NaN,2011.0,8.0,12.0


0      1572117.54
1      1807545.43
2      1644470.66
3      1857533.70
4       695396.19
          ...    
118     424513.08
119    2248645.59
120     845252.21
121     856796.10
122    1255087.26
Name: Weekly_Sales, Length: 123, dtype: float64

On va utiliser 80% de données en entrainement et 20% en test. On va utiliser le random_state à 20 pour pouvoir réutiliser la même répartition de données pour chaque test.

In [265]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=20)
print(X_train.shape)
print(X_test.shape)

(98, 10)
(25, 10)


## Pré processing
- Pour les valeurs numériques
  - On impute les valeurs moyennes aux valeur absentes
  - On effectue un StandardScaler()  => Cela permet de mettre à l'échelle les données numériques en fonction de la moyenne et de l'écart type
- Pour les variables catégorielles
  - Pas d'imputation de valeur manquante car Store et Holiday_Flag sont tous renseignés
  - OneHotEncoder permet de convertir les catégories en vecteur binaire et on supprime la première occurence afin d'éviter la colinéarité 

In [266]:
numerical_columns = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "Year", "Month", "Day"]
categorical_columns = ["Store", "Holiday_Flag"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(
    steps=[
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
    ])

feature_encoder = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_columns),
        ("cat", categorical_transformer, categorical_columns)
    ])

X_train = feature_encoder.fit_transform(X_train)
X_test = feature_encoder.transform(X_test) 

## calcul de la baseline

Pour la baseline, nous allons prendre en compte la moyenne qui au vu des graphiques précédents semble être la meilleure solution comme baseline

In [267]:
# Calculer la moyenne de la variable cible dans les données d'entraînement
mean_target = Y_train.mean()

# Prédire avec la moyenne pour toutes les instances de l'ensemble de test
Y_pred_baseline = [mean_target] * len(Y_test)

# Évaluer la baseline
rmse_baseline = np.sqrt(mean_squared_error(Y_test, Y_pred_baseline))
mae_baseline = mean_absolute_error(Y_test, Y_pred_baseline)

print(f"Baseline RMSE: {rmse_baseline:.2f}")
print(f"Baseline MAE: {mae_baseline:.2f}")


Baseline RMSE: 772664.75
Baseline MAE: 709060.65


## Entrainement d'un premier modèle de régression linéaire

In [268]:
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")


print("R2 score training :", regressor.score(X_train, Y_train))
print("R2 score test :",  regressor.score(X_test, Y_test))


print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")

print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")

# Évaluation du modèle
mse = mean_squared_error(Y_test, Y_test_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_test_pred)
r2_train = r2_score(Y_train, Y_train_pred)
r2_test = r2_score(Y_test, Y_test_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R² train: {r2_train}")
print(f"R² test: {r2_test}")

Train model...
...Done.
R2 score training : 0.9753304610344489
R2 score test : 0.9087736944954887
Predictions on training set...
...Done.
Predictions on test set...
...Done.
RMSE: 200864.33427095728
MAE: 146543.60371109901
R² train: 0.9753304610344489
R² test: 0.9087736944954887


## sauvegarde des scores

In [ ]:
scores_df = pd.DataFrame(columns = ["model", "R2_train", "R2_test","RMSE","MAE"])

new_rows = [{"model": "baseline", "RMSE":rmse_baseline,"MAE":mae_baseline}]
scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
new_rows = [{"model": "linear_regression", "R2_train": r2_train,"R2_test":r2_test,"RMSE":rmse,"MAE":mae}]
scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
scores_df.to_csv("../data/Walmart_Scores.csv", mode="w", index=False)


## validaiton croisée

In [270]:
scores = cross_val_score(regressor, X_train, Y_train, cv = 10)

print("The cross-validated R2-score is : ", scores.mean())
print("The standard deviation is : ", scores.std())

The cross-validated R2-score is :  0.9334061444440331
The standard deviation is :  0.04511234608506043


Ce premier modèle de régression linéaire est meilleur que la baseline mais il sur apprend, presque 0.07 points d'écart entre train et test.
L'écart-type obtenu avec la validation croisée montre une certaine stabilité.

## Analyse des coefficients.

In [ ]:
# --- Analyse des coefficients (robuste aux Pipelines/ColumnTransformer) ---

def get_ct_feature_names(ct: ColumnTransformer) -> list:
    """
    Reconstruit la liste des features en sortie d'un ColumnTransformer,
    en gérant les Pipelines et les OneHotEncoder de façon robuste.
    """
    output_features = []

    for name, transformer, cols in ct.transformers_:
        if transformer == "drop":
            continue
        if transformer == "passthrough":
            # Les colonnes passées telles quelles
            if isinstance(cols, (list, tuple, np.ndarray, pd.Index)):
                output_features.extend(list(cols))
            else:
                output_features.append(cols)
            continue

        # Si c'est un Pipeline, on prend l'objet final (le dernier step)
        final_est = transformer
        if isinstance(transformer, Pipeline):
            final_est = transformer.steps[-1][1]

        # Si l'estimateur final expose get_feature_names_out, on l'utilise
        if hasattr(final_est, "get_feature_names_out"):
            try:
                # cols peut être un Index/ndarray -> convertir en liste de str
                base_cols = list(cols) if isinstance(cols, (list, tuple, np.ndarray, pd.Index)) else [cols]
                names = final_est.get_feature_names_out(base_cols)
                output_features.extend(list(names))
                continue
            except Exception:
                # On tombera plus bas si ça échoue
                pass

        # Cas particulier OneHotEncoder trouvé au milieu du pipeline
        # (au cas où le dernier step n'expose pas les noms)
        if isinstance(final_est, OneHotEncoder):
            try:
                base_cols = list(cols) if isinstance(cols, (list, tuple, np.ndarray, pd.Index)) else [cols]
                names = final_est.get_feature_names_out(base_cols)
                output_features.extend(list(names))
                continue
            except Exception:
                pass

        # Fallback: si on ne sait pas, on renvoie les noms d'entrée
        # (utile si StandardScaler/Imputer sans OHE)
        if isinstance(cols, (list, tuple, np.ndarray, pd.Index)):
            output_features.extend(list(cols))
        else:
            output_features.append(cols)

    return output_features

# 1) Récupérer les noms de features après encodage
feature_names = get_ct_feature_names(feature_encoder)

# 2) Vérifier l'alignement taille coefficients <-> features
coefs = np.asarray(regressor.coef_).ravel()
if coefs.shape[0] != len(feature_names):
    raise ValueError(
        f"Nombre de coefficients ({coefs.shape[0]}) ≠ nombre de features ({len(feature_names)}). "
        "Vérifie que tu utilises les mêmes X_train transformés pour l'entraînement, "
        "et que le ColumnTransformer n'a pas changé entre fit et transform."
    )

# 3) DataFrame trié par importance absolue
coef_df = (
    pd.DataFrame({"feature": feature_names, "coef": coefs})
      .assign(abs_coef=lambda d: d["coef"].abs())
      .sort_values("abs_coef", ascending=False)
      .reset_index(drop=True)
)

# 4) (Optionnel) Visualisation rapide avec plotly
try:
    import plotly.express as px
    topN = 25
    fig = px.bar(
        coef_df.head(topN),
        x="abs_coef",
        y="feature",
        orientation="h",
        title=f"Top {topN} coefficients (|coef|)",
        labels={"abs_coef": "|coef|", "feature": "Feature"},
    )
    fig.update_layout(yaxis={"categoryorder": "total ascending"})
    fig.show()
except Exception as e:
    print("Plotly n'est pas disponible ou a échoué :", e)


On peut voir l'importance du magasin